In [1]:
import mesa
print("Mesa version:", getattr(mesa, "__version__", "unknown"))

Mesa version: 3.3.0


In [2]:
from __future__ import annotations
import mesa

class Walker(mesa.Agent):
    """Picks a random neighboring cell each step."""
    def __init__(self, model):
        super().__init__(model)  # auto-registers the agent with the model

    def step(self):
        neighborhood = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=True
        )
        new_pos = self.random.choice(neighborhood)  # Agent-scoped RNG
        self.model.grid.move_agent(self, new_pos)

class TinyWorld(mesa.Model):
    """10x10 torus with N random walkers."""
    def __init__(self, width=10, height=10, N=8, seed=None):
        super().__init__(seed=seed)
        self.grid = mesa.space.MultiGrid(width, height, torus=True)

        # create & place agents (auto-registered via Agent.__init__)
        for _ in range(N):
            a = Walker(self)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

    def step(self):
        # Random activation via AgentSet API (replaces RandomActivation)
        self.agents.shuffle_do("step")  # increments Model.steps automatically

In [3]:
world = TinyWorld(width=10, height=10, N=8, seed=42)

print("Initial positions (first 5 agents):")
for i, a in enumerate(list(world.agents)[:5]):
    print(i, a.unique_id, a.pos)

for _ in range(5):
    world.step()

print("\nAfter 5 steps (first 5 agents):")
for i, a in enumerate(list(world.agents)[:5]):
    print(i, a.unique_id, a.pos)

print("\nPopulation:", len(world.agents), "| Steps:", world.steps)

Initial positions (first 5 agents):
0 1 (1, 0)
1 2 (4, 3)
2 3 (3, 2)
3 4 (1, 8)
4 5 (1, 9)

After 5 steps (first 5 agents):
0 1 (8, 6)
1 2 (3, 5)
2 3 (2, 2)
3 4 (0, 6)
4 5 (3, 8)

Population: 8 | Steps: 5
